In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers



In [ ]:
#import data

df = pd.read_csv (r'track-1-essay-empathy-train.csv')
print (df)

               message_id  ... iri_empathatic_concern
0     R_1hGrPtWM4SumG0U_1  ...                  4.000
1     R_1hGrPtWM4SumG0U_2  ...                  4.000
2     R_1hGrPtWM4SumG0U_3  ...                  4.000
3     R_1hGrPtWM4SumG0U_4  ...                  4.000
4     R_1hGrPtWM4SumG0U_5  ...                  4.000
...                   ...  ...                    ...
1855  R_DHy2Rcz9Hym8jgl_1  ...                  4.429
1856  R_DHy2Rcz9Hym8jgl_2  ...                  4.429
1857  R_DHy2Rcz9Hym8jgl_3  ...                  4.429
1858  R_DHy2Rcz9Hym8jgl_4  ...                  4.429
1859  R_DHy2Rcz9Hym8jgl_5  ...                  4.429

[1860 rows x 22 columns]


In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['essay'], df['empathy_bin'], test_size=0.10,)


In [ ]:
max_length = max([len(s.split()) for s in df['essay']])

In [ ]:
print(max_length)

163


In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/MyDrive/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['essay'])
word_index = token.word_index

vocab_size = len(token.word_index) + 1


# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=max_length)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=max_length)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.0897      0.016      -0.0571     ...  0.1559     -0.0254
  -0.0259    ]
 [ 0.0495      0.0411      0.0041     ...  0.23710001 -0.0298
  -0.0284    ]
 ...
 [ 0.0114     -0.0305     -0.0284     ...  0.2199     -0.2313
   0.0016    ]
 [ 0.21969999 -0.1183     -0.0465     ...  0.0931     -0.0329
   0.1121    ]
 [-0.3256      0.0255      0.0938     ... -0.0964      0.0724
  -0.0605    ]]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding

In [ ]:
Embedding_dims = 300
model = Sequential()
model.add(Embedding(vocab_size, Embedding_dims, weights=[embedding_matrix], input_length=max_length, trainable=True,))
model.add(GlobalMaxPooling1D())
#model.add(GRU(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


    
    

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 163, 300)          3017100   
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 301       
Total params: 3,017,401
Trainable params: 3,017,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_seq_x, train_y, batch_size=128, epochs=20, validation_data=(valid_seq_x, valid_y), verbose=2)

Epoch 1/20
14/14 - 2s - loss: 0.6960 - accuracy: 0.4898 - val_loss: 0.6979 - val_accuracy: 0.4677
Epoch 2/20
14/14 - 1s - loss: 0.6741 - accuracy: 0.6392 - val_loss: 0.6938 - val_accuracy: 0.5161
Epoch 3/20
14/14 - 1s - loss: 0.6568 - accuracy: 0.7503 - val_loss: 0.6905 - val_accuracy: 0.6129
Epoch 4/20
14/14 - 1s - loss: 0.6389 - accuracy: 0.8590 - val_loss: 0.6877 - val_accuracy: 0.5699
Epoch 5/20
14/14 - 1s - loss: 0.6217 - accuracy: 0.8990 - val_loss: 0.6858 - val_accuracy: 0.5806
Epoch 6/20
14/14 - 1s - loss: 0.6037 - accuracy: 0.8787 - val_loss: 0.6842 - val_accuracy: 0.5430
Epoch 7/20
14/14 - 1s - loss: 0.5845 - accuracy: 0.8990 - val_loss: 0.6790 - val_accuracy: 0.6237
Epoch 8/20
14/14 - 1s - loss: 0.5627 - accuracy: 0.9863 - val_loss: 0.6769 - val_accuracy: 0.5860
Epoch 9/20
14/14 - 1s - loss: 0.5411 - accuracy: 0.9875 - val_loss: 0.6723 - val_accuracy: 0.6183
Epoch 10/20
14/14 - 1s - loss: 0.5172 - accuracy: 0.9875 - val_loss: 0.6690 - val_accuracy: 0.6129
Epoch 11/20
14/14 -

In [ ]:
y = model.predict(x=train_seq_x[:20])
for i in range(len(y)):
  #print()
  print(i, ' ', y[i][0])

0   0.1930444
1   0.82704175
2   0.24878475
3   0.13226953
4   0.32381982
5   0.34261495
6   0.14369982
7   0.21993467
8   0.1511386
9   0.58142936
10   0.90013814
11   0.83139455
12   0.7888224
13   0.82673717
14   0.18677709
15   0.87183034
16   0.80852884
17   0.13672954
18   0.25953096
19   0.12953767


In [ ]:
print(df['distress_bin'][:20])

0     1
1     1
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     0
10    1
11    0
12    1
13    0
14    1
15    1
16    1
17    0
18    1
19    1
Name: distress_bin, dtype: int64
